## eoAPI Demo:  Kahramanmaras earthquakes


### Background

This notebook will review the different [eoAPI](https://github.com/developmentseed/eoAPI) services using the latest Open data from Maxar acquired for the M7.8 and M7.5 Kahramanmaras earthquakes in Turkey on February 6, 2023.

More on the event: https://www.usgs.gov/news/featured-story/m78-and-m75-kahramanmaras-earthquake-sequence-near-nurdagi-turkey-turkiye

### Objective

This notebook aims to demonstrate how [eoAPI](https://github.com/developmentseed/eoAPI) can be used to analyze Maxar's high-resolution satellite data to assess the Kahramanmaras earthquakes' impact.

### Maxar Open Data

Pre and post-event high-resolution satellite imagery in support of emergency planning, risk assessment, monitoring of staging areas and emergency response, damage assessment, and recovery. These images are generated using the Maxar ARD pipeline, tiled on an organized grid in analysis-ready cloud-optimized formats.

### STAC and COGs

Maxar releases open data for select sudden-onset major crisis events. In addition to making the data (as nice COGs) freely available on AWS, they also add STAC (static) metadata alongside the images. Having the STAC items already created makes ingestion into the PgSTAC database easy (we don't have to produce the items ourselves and thus have to read the images).

To learn more about ingesting the Maxar OpenData STAC catalog into PgSTAC see https://github.com/vincentsarago/MAXAR_opendata_to_pgstac.

---

> 📣 **Note: Important Information**
> - **Demo Endpoints**: This notebook uses the demo endpoints [stac.eoapi.dev](https://stac.eoapi.dev) and [raster.eoapi.dev](https://raster.eoapi.dev). These are not intended for production use.
> - **Authentication**: While the demo endpoints are currently open to the public, we may introduce authentication features in the future.
> - **Set Up Your Own Services**: If you're interested in setting up your own eoAPI services, you can easily do so. For a step-by-step guide, please check out the 'Getting Started' section of the [eoAPI GitHub repository](https://github.com/developmentseed/eoAPI).
> - **Breaking changes**: DevelopmentSeed does not guarantee that these eoAPI endpoints will stay `open`, and breaking changes might be introduced anytime.

---

# STAC Metadata

Endpoint: https://stac.eoapi.dev

In [20]:
# Requirements

# !python -m pip install httpx ipyleaflet

In [55]:
from datetime import datetime

import json
import httpx

import ipyleaflet

### Collection

If you look in `https://stac.eoapi.dev/collections` response, you'll find one collection for the Kahramanmaras earthquake named `MAXAR_Kahramanmaras_turkey_earthquake_23`.

In [56]:
# list the collections and find the collection_id associated to the `kahramanmaras` event
collections = httpx.get("https://stac.eoapi.dev/collections").json()
print([c["id"] for c in collections["collections"] if "kahramanmaras" in c["description"].lower()])

['MAXAR_Kahramanmaras_turkey_earthquake_23']


Let's check the collection's metadata:

In [57]:
collection_id = "MAXAR_Kahramanmaras_turkey_earthquake_23"

collection_info = httpx.get(f"https://stac.eoapi.dev/collections/{collection_id}").json()

geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [bbox[0], bbox[1]],
                    [bbox[2], bbox[1]],
                    [bbox[2], bbox[3]],
                    [bbox[0], bbox[3]],
                    [bbox[0], bbox[1]],
                ]]
            },
            'properties': {}
        }
        for bbox in collection_info["extent"]["spatial"]["bbox"]
    ]
}

mainbbox = collection_info["extent"]["spatial"]["bbox"][0]

m = ipyleaflet.leaflet.Map(
    center=((mainbbox[1] + mainbbox[3]) / 2,(mainbbox[0] + mainbbox[2]) / 2),
    zoom=7
)

geo_json = ipyleaflet.leaflet.GeoJSON(data=geojson)
m.add_layer(geo_json)
m

Map(center=[36.983902384345036, 36.89273358165191], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Each collection can have spatial and temporal extents. As for the spatial extent, a collection can have multiple temporal extents, but its first one represents the combined min/max of all the intervals.

In [58]:
print(collection_info["extent"]["temporal"])

{'interval': [['2021-02-28T08:10:22Z', '2023-03-11T08:29:15Z']]}


## Items

In this section, we will:

- List all items for a specific collection using the `/collections/{collection_id}/items` endpoint
- Talk about the `limit` parameter
- Visualize all items on a map
- Talk about the item metadata
- List the Assets available for one Item

In [59]:
items = httpx.get(f"https://stac.eoapi.dev/collections/{collection_id}/items").json()
        
print(f"Nb Items in Db: {items['context']['matched']}")
print(f"Returned {len(items['features'])} Items")

Nb Items in Db: 2115
Returned 10 Items


As you can see below, the `/items` endpoints returned only 10 items. To return more data, we need to either use the `paging` mechanism.

In [60]:
kahramanmaras_items = []

url = f"https://stac.eoapi.dev/collections/{collection_id}/items"
while True:
    items = httpx.get(url, params={"limit": 200}).json()
    
    kahramanmaras_items.extend(items["features"])
    next_link = list(filter(lambda link: link["rel"] == "next", items["links"]))
    if next_link:
        url = next_link[0]["href"]
    else:
        break

print(f"Nb Items: {len(kahramanmaras_items)}")

Nb Items: 2115


In [61]:
m = ipyleaflet.leaflet.Map(
    center=((mainbbox[1] + mainbbox[3]) / 2,(mainbbox[0] + mainbbox[2]) / 2),
    zoom=7
)

event_date = datetime(2023, 2, 6, hour=0, minute=0)

# Use a styling function to show where we have before/after items
def style_function(feature):
    d = datetime.strptime(feature["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ")
    return {
        "fillOpacity": 0.1,
        "weight": 0.1,
        # Blue for pre-event / red for post-event
        "fillColor": "#0000ff" if d < event_date else "#ff0000"
    }

geo_json = ipyleaflet.leaflet.GeoJSON(data={"type": "FeatureCollection", "features": kahramanmaras_items}, style_callback=style_function)
m.add_layer(geo_json)
m

Map(center=[36.983902384345036, 36.89273358165191], controls=(ZoomControl(options=['position', 'zoom_in_text',…

#####  Item metadata 

Each item should have an `id`, a `geometry`, some links to `Assets`, and a set of properties.

Item specification: https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md

In [62]:
item = kahramanmaras_items[0]
print("Item example:")
print(json.dumps(item, indent=4))

Item example:
{
    "id": "37_031133210001_10300100E49E8000",
    "bbox": [
        36.387245559111236,
        36.092041667396806,
        36.42079643694594,
        36.11834923545021
    ],
    "type": "Feature",
    "links": [
        {
            "rel": "collection",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23"
        },
        {
            "rel": "parent",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23"
        },
        {
            "rel": "root",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/"
        },
        {
            "rel": "self",
            "type": "application/geo+json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23/items/37_031133210001_10300100E49E8000"
        }
    ],
    "assets": {
     

In [63]:
print("Item Id", item["id"])
print("Item Assets:", list(item["assets"].keys()))
print("Item Properties:")
print(json.dumps(item["properties"], indent=4))

Item Id 37_031133210001_10300100E49E8000
Item Assets: ['visual', 'data-mask', 'ms_analytic', 'pan_analytic']
Item Properties:
{
    "gsd": 0.48,
    "quadkey": "031133210001",
    "datetime": "2023-03-11T08:29:15Z",
    "platform": "WV02",
    "utm_zone": 37,
    "grid:code": "MXRA-Z37-031133210001",
    "proj:bbox": [
        264843.75,
        3997237.548828125,
        267866.2109375,
        4000156.25
    ],
    "proj:epsg": 32637,
    "catalog_id": "10300100E49E8000",
    "view:azimuth": 175.5,
    "proj:geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    264843.75,
                    4000156.25
                ],
                [
                    264843.75,
                    3997362.6708984375
                ],
                [
                    266686.70654296875,
                    3997294.921875
                ],
                [
                    267782.89794921875,
                    3997237.5

#### Find acquisition times

Every item should have either a `datetime` or a `start/end_datetime` property. For the Maxar dataset, we are assuming that `datetime` is acquisition times.

In [64]:
datetimes = {item["properties"]["datetime"] for item in kahramanmaras_items}
print("Dates:", sorted(list(datetimes)))

Dates: ['2021-02-28T08:10:22Z', '2021-02-28T08:10:23Z', '2021-08-17T11:16:54Z', '2021-08-18T08:13:11Z', '2021-08-18T08:13:12Z', '2021-09-09T08:36:18Z', '2021-09-09T08:36:19Z', '2021-09-09T08:36:20Z', '2021-09-27T08:40:11Z', '2021-09-27T08:40:12Z', '2021-09-27T08:40:58Z', '2021-09-27T08:41:51Z', '2021-09-27T08:41:52Z', '2021-09-27T08:41:53Z', '2021-09-27T08:41:54Z', '2022-06-18T08:14:17Z', '2022-06-18T08:14:18Z', '2022-07-20T08:39:24Z', '2022-07-20T08:39:25Z', '2022-07-20T08:39:37Z', '2022-07-20T08:39:38Z', '2022-07-26T08:19:46Z', '2022-08-05T08:51:08Z', '2022-08-05T08:51:09Z', '2022-10-10T08:19:25Z', '2022-10-10T08:19:26Z', '2022-10-17T08:44:40Z', '2022-10-17T08:44:59Z', '2022-10-31T08:44:00Z', '2022-10-31T08:44:01Z', '2022-12-16T08:45:41Z', '2022-12-16T08:45:42Z', '2022-12-22T08:24:28Z', '2022-12-22T08:24:29Z', '2022-12-27T08:40:06Z', '2022-12-27T08:40:07Z', '2022-12-27T08:40:17Z', '2023-01-02T08:18:54Z', '2023-01-02T08:18:55Z', '2023-01-02T08:19:11Z', '2023-01-07T08:39:58Z', '2023-01

Let's sort the items in two categories: `before` and `after` the event.

In [65]:
event_date = datetime(2023, 2, 6, hour=0, minute=0)

pre_items = list(
    filter(
        lambda item: datetime.strptime(item["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ") < event_date, 
        kahramanmaras_items
    )
)

post_items = list(
    filter(
        lambda item: datetime.strptime(item["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ") >= event_date, 
        kahramanmaras_items
    )
)
print("PRE event items:", len(pre_items))
print("POST event items:", len(post_items))

PRE event items: 229
POST event items: 1886


##### Other metadata

In [66]:
catalog_id = {item["properties"]["catalog_id"] for item in kahramanmaras_items}
print("Maxar Catalog Id:", sorted(list(catalog_id)))

platform = {item["properties"]["platform"] for item in kahramanmaras_items}
print("Maxar Platform: ", sorted(list(platform)))

Maxar Catalog Id: ['10200100B5C3A800', '10300100D6740900', '10300100D6789800', '10300100D76F1300', '10300100D797E100', '10300100DB913300', '10300100DC08C900', '10300100DD62DB00', '10300100DF069700', '10300100DF1DB000', '10300100DF6BFE00', '10300100DFA9DD00', '10300100E0287700', '10300100E131D000', '10300100E1700300', '10300100E18A1000', '10300100E18CB600', '10300100E19A4400', '10300100E1B9D900', '10300100E201BA00', '10300100E20A6300', '10300100E21A3000', '10300100E2226200', '10300100E2514200', '10300100E26DC400', '10300100E2780300', '10300100E287FA00', '10300100E2901100', '10300100E291D100', '10300100E29F9100', '10300100E2A73B00', '10300100E2B30500', '10300100E2BF8000', '10300100E2C66F00', '10300100E2D3B600', '10300100E304B400', '10300100E3154100', '10300100E32E3D00', '10300100E32F7A00', '10300100E383D900', '10300100E3CF1000', '10300100E3D52E00', '10300100E49E8000', '10300500BFF94800', '10300500BFF94C00', '10300500BFF95000', '10300500D8F90C00', '10300500D9F8CC00', '10300500D9F8D100', '

Note: The `catalog_id` represents a unique acquisition ID. In our case, this means that all items are derived from 5 different acquisitions. 

> The unique identifier for the item, as assigned by the provider, Maxar. This is also referred to as the "acquisition ID".

ref: https://ard.maxar.com/docs/select-and-order/select-stac-collection-file/#properties-object

## Asset visualization

So we have **2115** items for the `MAXAR_Kahramanmaras_turkey_earthquake_23` collection, and each item has **4** assets (this is also found at the collection level in the `item_assets extension`).

In [67]:
print(list(item["assets"].keys()))
print(json.dumps(item["assets"]["visual"], indent=4))

for name, asset in item["assets"].items():
    print(name, ": ", asset["type"])

['visual', 'data-mask', 'ms_analytic', 'pan_analytic']
{
    "href": "s3://maxar-opendata/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133210001/2023-03-11/10300100E49E8000-visual.tif",
    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
    "roles": [
        "visual"
    ],
    "title": "Visual Image",
    "eo:bands": [
        {
            "name": "BAND_R",
            "common_name": "red",
            "description": "Red"
        },
        {
            "name": "BAND_G",
            "common_name": "green",
            "description": "Green"
        },
        {
            "name": "BAND_B",
            "common_name": "blue",
            "description": "Blue"
        }
    ],
    "alternate": {
        "public": {
            "href": "https://maxar-opendata.s3.amazonaws.com/events/Kahramanmaras-turkey-earthquake-23/ard/37/031133210001/2023-03-11/10300100E49E8000-visual.tif",
            "title": "Public Access"
        }
    },
    "proj:bbox": [
       

In eoAPI, we have a raster API connected to the PgSTAC database. The service is built using [titiler-pgstac](http://github.com/stac-utils/titiler-pgstac) and can be used to visualize `Item` or `Mosaics` (multiple items).

Endpoint: [https://raster.eoapi.dev](https://raster.eoapi.dev/)

We know we have 4 assets for each item, and 3 are of `Cloud-Optimized` type. Let's use the raster API to visualize them.

First, let's get the Raster metadata for each `raster` asset. The raster API will use the asset's `type` metadata to filter non-raster dataset (e.g.  `application/geopackage+sqlite3`).

In [68]:
# fetching Raster information for all the `raster` assets
item_id = item["id"]

print(f"Fetching Raster info for Item {item_id}")
info = httpx.get(f"https://raster.eoapi.dev/collections/{collection_id}/items/{item_id}/info").json()

print("Returned metadata for Assets:", list(info.keys()))

print(json.dumps(info["visual"], indent=4))
for name, asset in info.items():
    print(name, asset["minzoom"], asset["maxzoom"])

Fetching Raster info for Item 37_031133210001_10300100E49E8000
Returned metadata for Assets: ['visual', 'ms_analytic', 'pan_analytic']
{
    "bounds": [
        36.387245559111236,
        36.069775171369855,
        36.447766515172646,
        36.11889420532333
    ],
    "minzoom": 12,
    "maxzoom": 19,
    "band_metadata": [
        [
            "b1",
            {}
        ],
        [
            "b2",
            {}
        ],
        [
            "b3",
            {}
        ]
    ],
    "band_descriptions": [
        [
            "b1",
            ""
        ],
        [
            "b2",
            ""
        ],
        [
            "b3",
            ""
        ]
    ],
    "dtype": "uint8",
    "nodata_type": "Mask",
    "colorinterp": [
        "red",
        "green",
        "blue"
    ],
    "driver": "GTiff",
    "count": 3,
    "width": 17408,
    "height": 17408,
    "overviews": [
        2,
        4,
        8,
        16,
        32,
        64
    ]
}
visual 

The `/collections/{collectionId}/items/{itemId}/info` endpoint returned metadata for 3 assets (the raster ones). We now know more about each asset (datatype, zoom levels, number of bands), which can help us create tiles urls.

### Asset on Map

To visualize an asset on a Map, we need to construct a `Tile URL`. To ease the task we can use the raster's service `/collections/{collection_id}/items/{item_id}/tilejson.json` endpoint, but here are the requirements:

- HAVE TO pass `assets` or `expression` parameter
- CAN pass `min/max zooms` (which will avoid under/over-zooming)
- CAN pass `rescale` parameter if datatype is not compatible with PNG/JPEG output format
- CAN pass `asset_bidx` parameter to select band combination

In [69]:
# `visual` Asset
tilejson = httpx.get(
    f"https://raster.eoapi.dev/collections/{collection_id}/items/{item_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS PARAMETER IS MANDATORY
        ("minzoom", 12),  # By default the tiler will use 0
        ("maxzoom", 19), # By default the tiler will use 24
    )
).json()
print(tilejson)

bounds = tilejson["bounds"]
m = ipyleaflet.leaflet.Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=12
)

geo_json = ipyleaflet.leaflet.GeoJSON(
    data=item,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0., 'weight': 4
    }
)
m.add_layer(geo_json)

tiles = ipyleaflet.leaflet.TileLayer(
    url=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],

    ],
)

m.add_layer(tiles)

m

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://raster.eoapi.dev/collections/MAXAR_Kahramanmaras_turkey_earthquake_23/items/37_031133210001_10300100E49E8000/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=visual'], 'minzoom': 12, 'maxzoom': 19, 'bounds': [36.387245559111236, 36.092041667396806, 36.42079643694594, 36.11834923545021], 'center': [36.404020998028585, 36.10519545142351, 12]}


Map(center=[36.10519545142351, 36.404020998028585], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Mosaics

As mentioned and shown with `titiler-pgstac`, we can visualize an item's asset. Still, the raster API's real power is to create a virtual mosaic dynamically and merge multiple items on the fly.

Learn more: http://github.com/stac-utils/titiler-pgstac.

Let's create `mosaics` for `pre` and `post` events.

In [70]:
event_date = "2023-02-06T00:00:00Z"

# Let's focus on Kahramanmaraş city 
bounds = [36.83064386785452, 37.53123515817725, 37.03859654890988, 37.63167525356958]

pre_mosaic = httpx.post(
    "https://raster.eoapi.dev/searches/register",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "in", 
                        "args": [{"property": "collection"}, [collection_id]]
                    },
                    {
                        "op": "lt", 
                        "args": [
                            {"property": "datetime"}, event_date
                        ]
                    }
                ],
            },
            "sortby": [
                {
                    "field": "tile:clouds_percent",
                    "direction": "asc"
                },
            ],
            "metadata":{
                "name": "Maxar Kahramanmaras - Pre event",
                "bounds": bounds,
            }
            
        }
    )
).json()

post_mosaic = httpx.post(
    "https://raster.eoapi.dev/searches/register",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "in", 
                        "args": [{"property": "collection"}, [collection_id]]
                    },
                    {
                        "op": "ge", 
                        "args": [
                            {"property": "datetime"}, event_date
                        ]
                    }
                ],
            },
            "sortby": [
                {
                    "field": "tile:clouds_percent",
                    "direction": "asc"
                },
            ],
            "metadata":{
                "name": "Maxar Kahramanmaras - Port event",
                "bounds": bounds,
            }
            
        }
    )
).json()

print("Pre event Mosaic")
print(json.dumps(pre_mosaic, indent=4))
print()
print("Post event Mosaic")
print(json.dumps(post_mosaic, indent=4))

Pre event Mosaic
{
    "id": "9c2101bb56005850dcc6e95303586432",
    "links": [
        {
            "rel": "metadata",
            "title": "Mosaic metadata",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/9c2101bb56005850dcc6e95303586432/info"
        },
        {
            "rel": "tilejson",
            "title": "Link for TileJSON",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/9c2101bb56005850dcc6e95303586432/tilejson.json"
        },
        {
            "rel": "map",
            "title": "Link for Map viewer",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/9c2101bb56005850dcc6e95303586432/map"
        },
        {
            "rel": "wmts",
            "title": "Link for WMTS",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/9c2101bb56005850dcc6e95303586432/WMTSCapabilities.xml"
        

Explanation:

The `mosaic` corresponds to an STAC Search query. Because we use `PgSTAC` backend, we can make use of the `filter` extension to construct complex queries: https://github.com/stac-api-extensions/filter

```json
{
    // PgSTAC accepts multiple languages for filtering; here we will use cql2-json
    "filter-lang": 'cql2-json',
    // We tell PgSTAC to `register` a `search` request with the following filter:
    "filter": {
        "op": 'and',
        "args": [
            // Item's collection HAS TO be in `[collection_id]`
            {
                "op": "in",
                "args": [{"property": "collection"}, [collection_id]]
            },
            // Filter Items that have datetime `lt` than the event date
            {
                "op": "lt",
                "args": [
                    {"property": "datetime"}, event_date
                ]
            },
            // Sort the items using clouds_percent property to make sure cloudless images are on the top
            "sortby": [
                {
                    "field": "tile:clouds_percent",
                    "direction": "asc"
                },
            ],
            // titiler-pgstac accept some additional metadata
            // <https://stac-utils.github.io/titiler-pgstac/advanced/metadata/>
            // One is useful: `bounds`. When creating a mosaic, the tiler will have no idea
            // where the items will be before trying to create each tile. To avoid trying to request
            // tiles where we know we don't have any items, we can add the collection's extent to the
            // mosaic metadata.
            // The tiler service will then return the bounds in the tilejson document for the
            // client application.
            "metadata":{
                "name": "Maxar Kahramanmaras - Pre event",
                "bounds": bounds,
            }
        ],
    },

}

```

API Response:

The raster service will return a `id` hash (`mosaic id`), which we can use to construct a tile URL.

To create a valid tile URL, we will again need to pass an `assets` parameter to tell the tiler which assets we want to visualize. We can also set the min/max zoom limits to avoid underzooming (opening too many files) and overzooming.

See the complete list of options: https://stac-utils.github.io/titiler-pgstac/mosaic_endpoints/#tiles

In [71]:
mosaic_id = pre_mosaic["id"]

tilejson_pre = httpx.get(
    f"https://raster.eoapi.dev/searches/{mosaic_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS IS MANDATORY
        ("minzoom", 12),
        ("maxzoom", 19), 
    )
).json()
print(tilejson_pre)

bounds = tilejson_pre["bounds"]
m = ipyleaflet.leaflet.Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=12
)

geo_json = ipyleaflet.leaflet.GeoJSON(
    data={"type": "FeatureCollection", "features": pre_items}, 
    style={
        "fillOpacity": 0,
        "weight": 1,
    },
)
m.add_layer(geo_json)

tiles = ipyleaflet.leaflet.TileLayer(
    url=tilejson_pre["tiles"][0],
    min_zoom=tilejson_pre["minzoom"],
    max_zoom=tilejson_pre["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],

    ],
)

m.add_layer(tiles)
m

{'tilejson': '2.2.0', 'name': 'Maxar Kahramanmaras - Pre event', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://raster.eoapi.dev/searches/9c2101bb56005850dcc6e95303586432/tiles/WebMercatorQuad/{z}/{x}/{y}?assets=visual'], 'minzoom': 12, 'maxzoom': 19, 'bounds': [36.83064386785452, 37.53123515817725, 37.03859654890988, 37.63167525356958], 'center': [36.9346202083822, 37.58145520587342, 12]}


Map(center=[37.58145520587342, 36.9346202083822], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [72]:
mosaic_id = post_mosaic["id"]

tilejson_post = httpx.get(
    f"https://raster.eoapi.dev/searches/{mosaic_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS IS MANDATORY
        ("minzoom", 12),
        ("maxzoom", 19), 
    )
).json()
print(tilejson_post)

bounds = tilejson_post["bounds"]
m = ipyleaflet.leaflet.Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=10
)

geo_json = ipyleaflet.leaflet.GeoJSON(
    data={"type": "FeatureCollection", "features": post_items}, 
    style={
        "fillOpacity": 0,
        "weight": 0.5,
    },
)
m.add_layer(geo_json)

tiles = ipyleaflet.leaflet.TileLayer(
    url=tilejson_post["tiles"][0],
    min_zoom=tilejson_post["minzoom"],
    max_zoom=tilejson_post["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],

    ],
)

m.add_layer(tiles)
m

{'tilejson': '2.2.0', 'name': 'Maxar Kahramanmaras - Port event', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://raster.eoapi.dev/searches/8aebe8860a92d2f7bc0cdad4d7d81acf/tiles/WebMercatorQuad/{z}/{x}/{y}?assets=visual'], 'minzoom': 12, 'maxzoom': 19, 'bounds': [36.83064386785452, 37.53123515817725, 37.03859654890988, 37.63167525356958], 'center': [36.9346202083822, 37.58145520587342, 12]}


Map(center=[37.58145520587342, 36.9346202083822], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [73]:
m = ipyleaflet.leaflet.Map(
    center=(37.571788, 36.919100),
    zoom=12,
)

bounds = tilejson_pre["bounds"]
before_layer = ipyleaflet.leaflet.TileLayer(
    url=tilejson_pre["tiles"][0],
    min_zoom=tilejson_pre["minzoom"],
    max_zoom=tilejson_pre["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],
    ],
)

bounds = tilejson_post["bounds"]
after_layer = ipyleaflet.leaflet.TileLayer(
    url=tilejson_post["tiles"][0],
    min_zoom=tilejson_post["minzoom"],
    max_zoom=tilejson_post["maxzoom"],
    bounds=[
        [bounds[1], bounds[0]],
        [bounds[3], bounds[2]],
    ],
)

control = ipyleaflet.leaflet.SplitMapControl(left_layer=before_layer, right_layer=after_layer)
m.add_control(control)

m

Map(center=[37.571788, 36.9191], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

## What's Next?

### Spin Up Your Own eoAPI Instance

You've seen what eoAPI can do with Maxar data in the context of the Turkey Earthquakes. Interested in setting up your own eoAPI service? It's straightforward! Follow the 'Getting Started' section of the [eoAPI GitHub repository](https://github.com/developmentseed/eoAPI) to get your instance up and running. This will give you greater control and customization options.

### Contribute Your Data

Consider contributing if you've used Maxar data for similar analyses or have other datasets that could benefit the community. Uploading your data to your eoAPI instance can provide more diverse examples and help in various applications.

### Community-Driven Examples

The examples you see here, including this notebook, are often community-driven. We encourage you to contribute your analyses, workflows, or visualizations. Your insights could be invaluable for helping others.

### Have Questions? Join the Discussion!

If you have questions, feedback, or want to engage with the eoAPI community, please join the [GitHub discussions](https://github.com/developmentseed/eoAPI/discussions). It's a great place to ask questions, share your experiences, and connect with others interested in eoAPI and its components. 

---

Thank you for taking the time to go through this notebook.